
<a href="https://github.com/groda/big_data"><div><img src="https://github.com/groda/big_data/blob/master/logo_bdb.png?raw=true" align=right width="90" alt="Logo Big Data for Beginners"></div></a>
# Apache Sedona with PySpark

Apache Sedona™ is

> *a cluster computing system for processing large-scale spatial data. Sedona extends existing cluster computing systems, such as Apache Spark, Apache Flink, and Snowflake, with a set of out-of-the-box distributed Spatial Datasets and Spatial SQL that efficiently load, process, and analyze large-scale spatial data across machines.* ([https://sedona.apache.org/](https://sedona.apache.org/))

To execute a basic Sedona demonstration using PySpark on Google Colab, we made a few minor adjustments. The Sedona notebook starts below at [Apache Sedona Core demo](#scrollTo=Apache_Sedona_Core_demo).



## Install Apache Sedona and PySpark

To start with, we are going to install PySpark with Sedona following the instructions at: https://sedona.apache.org/latest-snapshot/setup/install-python/ but first we need to downgrade `shapely` because the version 2.0.2 that comes with Google Colab does not play well with the current version of Apache Sedona (see https://shapely.readthedocs.io/en/stable/migration.html).

### Downgrade Shapely to version 1.7.1

We need to install install any version of `shapely>=1.7.0` but smaller than `2.0`. We picked `1.7.1` because with 1.7.0 we got the error

    geopandas 0.13.2 requires shapely>=1.7.1, but you have shapely 1.7.0 which is incompatible.

Explanation for `pip -I`:

- [`-I, --ignore-installed`](https://pip.pypa.io/en/stable/cli/pip_install/#cmdoption-I)
> Ignore the installed packages, overwriting them. This can break your system if the existing package is of a different version or was installed with a different package manager!







In [1]:
!pip install -I shapely==1.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for shapely: filename=Shapely-1.7.1-cp310-cp310-linux_x86_64.whl size=997453 sha256=67cceca8f66108b0c8a04d63f39f311a9ff78500c22dc63eba4aacd8d170a81c
  Stored in directory: /root/.cache/pip/wheels/2e/fa/97/c85f587c35afcaf4a81c481741d36592518d1e50445572f0d4
Successfully built shapely


### Install Geopandas

This step is only needed outside of Colab because on Google Colab `geopandas` is available by default.

In [2]:
!pip install geopandas==0.13.2

### Install Apache Sedona and PySpark

We can now install Apache Sedona together with PySpark (and Spark).

In [3]:
!pip install apache-sedona[spark]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=03da787088e5210fe57ad3e14c480864ef5da4ff357193d76887d6ea20e9e34b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
%env SPARK_HOME = "/usr/local/lib/python3.10/dist-packages/pyspark"

env: SPARK_HOME="/usr/local/lib/python3.10/dist-packages/pyspark"


In [5]:
%env PYTHONPATH = /usr/local/lib/python3.10/dist-packages/pyspark/python

env: PYTHONPATH=/usr/local/lib/python3.10/dist-packages/pyspark/python


In [6]:
!pip info pyspark

ERROR: unknown command "info"


## Setup environment variables

We need to set two environment variables:

- `SPARK_HOME`
- `PYTHONPATH`

Once we have set `SPARK_HOME`, the variable `PYTHONPATH` is `$SPARK_HOME/python`.

### Find Spark home

There's an utility to find Spark home and I always forget how it's called exactly, what I remember is that it contains `"find"` and `"spark"`. Let us search for it:

In [7]:
!find / -name "*find*spark*"

find: ‘/proc/58/task/58/net’: Invalid argument
find: ‘/proc/58/net’: Invalid argument
/usr/local/bin/find_spark_home.py
/usr/local/bin/find-spark-home
/usr/local/bin/__pycache__/find_spark_home.cpython-310.pyc
/usr/local/bin/find-spark-home.cmd
/usr/local/lib/python3.10/dist-packages/pyspark/bin/find-spark-home
/usr/local/lib/python3.10/dist-packages/pyspark/bin/find-spark-home.cmd
/usr/local/lib/python3.10/dist-packages/pyspark/find_spark_home.py
/usr/local/lib/python3.10/dist-packages/pyspark/__pycache__/find_spark_home.cpython-310.pyc


The script `/usr/local/bin/find_spark_home.py` is successful at finding Spark's home.

#### Set `SPARK_HOME`

In [8]:
import sys
import os
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  output = !python /usr/local/bin/find_spark_home.py
else:
  output = !find / -name "pyspark" -type d 2>/dev/null|head -1
# Store the output using %store
%store output
# get rid of extra quotation marks
os.environ['SPARK_HOME'] = output[0].replace('"', '')

Stored 'output' (SList)


In [9]:
!pip show pyspark

Name: pyspark
Version: 3.5.1
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: py4j
Required-by: 


Verify that the correct `SPARK_HOME` has been set.

In [10]:
os.environ['SPARK_HOME']

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [11]:
%env SPARK_HOME

'/usr/local/lib/python3.10/dist-packages/pyspark'

#### Set `PYTHONPATH`

In [12]:
os.environ['PYTHONPATH'] = os.environ['SPARK_HOME'] + '/python'

Check

In [13]:
%env PYTHONPATH

'/usr/local/lib/python3.10/dist-packages/pyspark/python'

## Download data

In order to run, the Sedona notebook expects to find some specific files in the local folder `data`. Let us populate `data` with the files from the Sedona Github repository.

In [18]:
# it would be more efficient to just download the "data" folder and not the whole repo
![ -d sedona ] || git clone https://github.com/apache/sedona.git

!cp -r sedona/binder/data ./


cp: cannot stat 'sedona/binder/data': No such file or directory


Verify the presence of data in the designated `data` folder.

In [19]:
!ls data

ls: cannot access 'data': No such file or directory


# Apache Sedona Core demo

The notebook is available at the following link:
https://github.com/apache/sedona/blob/master/binder/ApacheSedonaCore.ipynb


```
Licensed to the Apache Software Foundation (ASF) under one
or more contributor license agreements.  See the NOTICE file
distributed with this work for additional information
regarding copyright ownership.  The ASF licenses this file
to you under the Apache License, Version 2.0 (the
"License"); you may not use this file except in compliance
with the License.  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied.  See the License for the
specific language governing permissions and limitations
under the License.
```

In [20]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import LongType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.spark import *
from sedona.core.geom.envelope import Envelope


Skipping SedonaKepler import, verify if keplergl is installed
Skipping SedonaPyDeck import, verify if pydeck is installed


In [21]:
config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.4_2.12:1.5.1,'
           'org.datasyslab:geotools-wrapper:1.5.1-28.2,'
           'uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.4'). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all'). \
    getOrCreate()

sedona = SedonaContext.create(config)

In [22]:
sc = sedona.sparkContext

# Create SpatialRDD

## Reading to PointRDD from CSV file

Suppose we want load the CSV file into Apache Sedona PointRDD
```
testattribute0,-88.331492,32.324142,testattribute1,testattribute2
testattribute0,-88.175933,32.360763,testattribute1,testattribute2
testattribute0,-88.388954,32.357073,testattribute1,testattribute2
testattribute0,-88.221102,32.35078,testattribute1,testattribute2
testattribute0,-88.323995,32.950671,testattribute1,testattribute2
testattribute0,-88.231077,32.700812,testattribute1,testattribute2
```

In [23]:
point_rdd = PointRDD(sc, "data/arealm-small.csv", 1, FileDataSplitter.CSV, True, 10)

Py4JJavaError: An error occurred while calling None.org.apache.sedona.core.spatialRDD.PointRDD.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/content/data/arealm-small.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPa

In [ ]:
## Getting approximate total count
point_rdd.approximateTotalCount

In [ ]:
# getting boundary for PointRDD or any other SpatialRDD, it returns Enelope object which inherits from
# shapely.geometry.Polygon
point_rdd.boundary()

In [ ]:
# To run analyze please use function analyze
point_rdd.analyze()

In [ ]:
# Finding boundary envelope for PointRDD or any other SpatialRDD, it returns Enelope object which inherits from
# shapely.geometry.Polygon
point_rdd.boundaryEnvelope

In [ ]:
# Calculate number of records without duplicates
point_rdd.countWithoutDuplicates()

In [ ]:
# Getting source epsg code
point_rdd.getSourceEpsgCode()

In [ ]:
# Getting target epsg code
point_rdd.getTargetEpsgCode()

In [ ]:
# Spatial partitioning data
point_rdd.spatialPartitioning(GridType.KDBTREE)

## Operations on RawSpatialRDD

rawSpatialRDD method returns RDD which consists of GeoData objects which has 2 attributes
<li> geom: shapely.geometry.BaseGeometry </li>
<li> userData: str </li>

You can use any operations on those objects and spread across machines

In [ ]:
# take firs element
point_rdd.rawSpatialRDD.take(1)

In [ ]:
# collect to Python list
point_rdd.rawSpatialRDD.collect()[:5]

In [ ]:
# apply map functions, for example distance to Point(52 21)
point_rdd.rawSpatialRDD.map(lambda x: x.geom.distance(Point(21, 52))).take(5)

## Transforming to GeoPandas

## Loaded data can be transformed to GeoPandas DataFrame in a few ways

### Directly from RDD

In [ ]:
point_rdd_to_geo = point_rdd.rawSpatialRDD.map(lambda x: [x.geom, *x.getUserData().split("\t")])

In [ ]:
point_gdf = gpd.GeoDataFrame(
    point_rdd_to_geo.collect(), columns=["geom", "attr1", "attr2", "attr3"], geometry="geom"
)

In [ ]:
point_gdf[:5]

### Using Adapter

In [ ]:
# Adapter allows you to convert geospatial data types introduced with sedona to other ones

In [ ]:
spatial_df = Adapter.\
    toDf(point_rdd, ["attr1", "attr2", "attr3"], sedona).\
    createOrReplaceTempView("spatial_df")

spatial_gdf = sedona.sql("Select attr1, attr2, attr3, geometry as geom from spatial_df")

In [ ]:
spatial_gdf.show(5, False)

In [ ]:
gpd.GeoDataFrame(spatial_gdf.toPandas(), geometry="geom")[:5]

### With DataFrame creation

In [ ]:
schema = StructType(
    [
        StructField("geometry", GeometryType(), False),
        StructField("attr1", StringType(), False),
        StructField("attr2", StringType(), False),
        StructField("attr3", StringType(), False),
    ]
)

In [ ]:
geo_df = sedona.createDataFrame(point_rdd_to_geo, schema, verifySchema=False)

In [ ]:
gpd.GeoDataFrame(geo_df.toPandas(), geometry="geometry")[:5]

# Load Typed SpatialRDDs

Currently The library supports 5 typed SpatialRDDs:
<li> RectangleRDD </li>
<li> PointRDD </li>
<li> PolygonRDD </li>
<li> LineStringRDD </li>
<li> CircleRDD </li>

In [ ]:
rectangle_rdd = RectangleRDD(sc, "data/zcta510-small.csv", FileDataSplitter.CSV, True, 11)
point_rdd = PointRDD(sc, "data/arealm-small.csv", 1, FileDataSplitter.CSV, False, 11)
polygon_rdd = PolygonRDD(sc, "data/primaryroads-polygon.csv", FileDataSplitter.CSV, True, 11)
linestring_rdd = LineStringRDD(sc, "data/primaryroads-linestring.csv", FileDataSplitter.CSV, True)

In [ ]:
rectangle_rdd.analyze()
point_rdd.analyze()
polygon_rdd.analyze()
linestring_rdd.analyze()

# Spatial Partitioning

Apache Sedona spatial partitioning method can significantly speed up the join query. Three spatial partitioning methods are available: KDB-Tree, Quad-Tree and R-Tree. Two SpatialRDD must be partitioned by the same way.

In [ ]:
point_rdd.spatialPartitioning(GridType.KDBTREE)

# Create Index

Apache Sedona provides two types of spatial indexes, Quad-Tree and R-Tree. Once you specify an index type, Apache Sedona will build a local tree index on each of the SpatialRDD partition.

In [ ]:
point_rdd.buildIndex(IndexType.RTREE, True)

# SpatialJoin

Spatial join is operation which combines data based on spatial relations like:
<li> intersects </li>
<li> touches </li>
<li> within </li>
<li> etc </li>

To Use Spatial Join in GeoPyspark library please use JoinQuery object, which has implemented below methods:
```python
SpatialJoinQuery(spatialRDD: SpatialRDD, queryRDD: SpatialRDD, useIndex: bool, considerBoundaryIntersection: bool) -> RDD

DistanceJoinQuery(spatialRDD: SpatialRDD, queryRDD: SpatialRDD, useIndex: bool, considerBoundaryIntersection: bool) -> RDD

spatialJoin(queryWindowRDD: SpatialRDD, objectRDD: SpatialRDD, joinParams: JoinParams) -> RDD

DistanceJoinQueryFlat(spatialRDD: SpatialRDD, queryRDD: SpatialRDD, useIndex: bool, considerBoundaryIntersection: bool) -> RDD

SpatialJoinQueryFlat(spatialRDD: SpatialRDD, queryRDD: SpatialRDD, useIndex: bool, considerBoundaryIntersection: bool) -> RDD

```

## Example SpatialJoinQueryFlat PointRDD with RectangleRDD

In [ ]:
# partitioning the data
point_rdd.spatialPartitioning(GridType.KDBTREE)
rectangle_rdd.spatialPartitioning(point_rdd.getPartitioner())
# building an index
point_rdd.buildIndex(IndexType.RTREE, True)
# Perform Spatial Join Query
result = JoinQuery.SpatialJoinQueryFlat(point_rdd, rectangle_rdd, False, True)

As result we will get RDD[GeoData, GeoData]
It can be used like any other Python RDD. You can use map, take, collect and other functions  

In [ ]:
result

In [ ]:
result.take(2)

In [ ]:
result.collect()[:3]

In [ ]:
# getting distance using SpatialObjects
result.map(lambda x: x[0].geom.distance(x[1].geom)).take(5)

In [ ]:
# getting area of polygon data
result.map(lambda x: x[0].geom.area).take(5)

In [ ]:
# Base on result you can create DataFrame object, using map function and build DataFrame from RDD

In [ ]:
schema = StructType(
    [
        StructField("geom_left", GeometryType(), False),
        StructField("geom_right", GeometryType(), False)
    ]
)

In [ ]:
# Set verifySchema to False
spatial_join_result = result.map(lambda x: [x[0].geom, x[1].geom])
sedona.createDataFrame(spatial_join_result, schema, verifySchema=False).show(5, True)

In [ ]:
# Above code produces DataFrame with geometry Data type

In [ ]:
sedona.createDataFrame(spatial_join_result, schema, verifySchema=False).printSchema()

We can create DataFrame object from Spatial Pair RDD using Adapter object as follows

In [ ]:
Adapter.toDf(result, ["attr1"], ["attr2"], sedona).show(5, True)

This also produce DataFrame with geometry DataType

In [ ]:
Adapter.toDf(result, ["attr1"], ["attr2"], sedona).printSchema()

We can create RDD which will be of type RDD[GeoData, List[GeoData]]
We can for example calculate number of Points within some polygon data

To do that we can use code specified below

In [ ]:
point_rdd.spatialPartitioning(GridType.KDBTREE)
rectangle_rdd.spatialPartitioning(point_rdd.getPartitioner())

In [ ]:
spatial_join_result_non_flat = JoinQuery.SpatialJoinQuery(point_rdd, rectangle_rdd, False, True)

In [ ]:
# number of point for each polygon
number_of_points = spatial_join_result_non_flat.map(lambda x: [x[0].geom, x[1].__len__()])

In [ ]:
schema = StructType([
    StructField("geometry", GeometryType(), False),
    StructField("number_of_points", LongType(), False)
])

In [ ]:
sedona.createDataFrame(number_of_points, schema, verifySchema=False).show()

# KNNQuery

Spatial KNNQuery is operation which help us find answer which k number of geometries lays closest to other geometry.

For Example:
    5 closest Shops to your home. To use Spatial KNNQuery please use object
<b> KNNQuery </b> which has one method:
```python
SpatialKnnQuery(spatialRDD: SpatialRDD, originalQueryPoint: BaseGeometry, k: int,  useIndex: bool)-> List[GeoData]
```

### Finds 5 closest points from PointRDD to given Point

In [ ]:
result = KNNQuery.SpatialKnnQuery(point_rdd, Point(-84.01, 34.01), 5, False)

In [ ]:
result

As Reference geometry you can also use Polygon or LineString object

In [ ]:
polygon = Polygon(
    [(-84.237756, 33.904859), (-84.237756, 34.090426),
     (-83.833011, 34.090426), (-83.833011, 33.904859),
     (-84.237756, 33.904859)
    ])
polygons_nearby = KNNQuery.SpatialKnnQuery(polygon_rdd, polygon, 5, False)

In [ ]:
polygons_nearby

In [ ]:
polygons_nearby[0].geom.wkt

# RangeQuery

A spatial range query takes as input a range query window and an SpatialRDD and returns all geometries that intersect / are fully covered by the query window.
RangeQuery has one method:

```python
SpatialRangeQuery(self, spatialRDD: SpatialRDD, rangeQueryWindow: BaseGeometry, considerBoundaryIntersection: bool, usingIndex: bool) -> RDD
```

In [ ]:
from sedona.core.geom.envelope import Envelope

In [ ]:
query_envelope = Envelope(-85.01, -60.01, 34.01, 50.01)

result_range_query = RangeQuery.SpatialRangeQuery(linestring_rdd, query_envelope, False, False)

In [ ]:
result_range_query

In [ ]:
result_range_query.take(6)

In [ ]:
# Creating DataFrame from result

In [ ]:
schema = StructType([StructField("geometry", GeometryType(), False)])

In [ ]:
sedona.createDataFrame(
    result_range_query.map(lambda x: [x.geom]),
    schema,
    verifySchema=False
).show(5, True)

# Load From other Formats

GeoPyspark allows to load the data from other Data formats like:
<li> GeoJSON </li>
<li> Shapefile </li>
<li> WKB </li>
<li> WKT </li>

In [ ]:
## ShapeFile - load to SpatialRDD

In [ ]:
shape_rdd = ShapefileReader.readToGeometryRDD(sc, "data/polygon")

In [ ]:
shape_rdd

In [ ]:
Adapter.toDf(shape_rdd, sedona).show(5, True)

In [ ]:
## GeoJSON - load to SpatialRDD

```
{ "type": "Feature", "properties": { "STATEFP": "01", "COUNTYFP": "077", "TRACTCE": "011501", "BLKGRPCE": "5", "AFFGEOID": "1500000US010770115015", "GEOID": "010770115015", "NAME": "5", "LSAD": "BG", "ALAND": 6844991, "AWATER": 32636 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ -87.621765, 34.873444 ], [ -87.617535, 34.873369 ], [ -87.6123, 34.873337 ], [ -87.604049, 34.873303 ], [ -87.604033, 34.872316 ], [ -87.60415, 34.867502 ], [ -87.604218, 34.865687 ], [ -87.604409, 34.858537 ], [ -87.604018, 34.851336 ], [ -87.603716, 34.844829 ], [ -87.603696, 34.844307 ], [ -87.603673, 34.841884 ], [ -87.60372, 34.841003 ], [ -87.603879, 34.838423 ], [ -87.603888, 34.837682 ], [ -87.603889, 34.83763 ], [ -87.613127, 34.833938 ], [ -87.616451, 34.832699 ], [ -87.621041, 34.831431 ], [ -87.621056, 34.831526 ], [ -87.62112, 34.831925 ], [ -87.621603, 34.8352 ], [ -87.62158, 34.836087 ], [ -87.621383, 34.84329 ], [ -87.621359, 34.844438 ], [ -87.62129, 34.846387 ], [ -87.62119, 34.85053 ], [ -87.62144, 34.865379 ], [ -87.621765, 34.873444 ] ] ] } },
```

In [ ]:
geo_json_rdd = GeoJsonReader.readToGeometryRDD(sc, "data/testPolygon.json")

In [ ]:
geo_json_rdd

In [ ]:
Adapter.toDf(geo_json_rdd, sedona).drop("AWATER").show(5, True)

In [ ]:
## WKT - loading to SpatialRDD

In [ ]:
wkt_rdd = WktReader.readToGeometryRDD(sc, "data/county_small.tsv", 0, True, False)

In [ ]:
wkt_rdd

In [ ]:
Adapter.toDf(wkt_rdd, sedona).printSchema()

In [ ]:
Adapter.toDf(wkt_rdd, sedona).show(5, True)

In [ ]:
## WKB - load to SpatialRDD

In [ ]:
wkb_rdd = WkbReader.readToGeometryRDD(sc, "data/county_small_wkb.tsv", 0, True, False)

In [ ]:
Adapter.toDf(wkb_rdd, sedona).show(5, True)

## Converting RDD Spatial join result to DF directly, avoiding jvm python serde

In [ ]:
point_rdd.spatialPartitioning(GridType.KDBTREE)
rectangle_rdd.spatialPartitioning(point_rdd.getPartitioner())
# building an index
point_rdd.buildIndex(IndexType.RTREE, True)
# Perform Spatial Join Query
result = JoinQueryRaw.SpatialJoinQueryFlat(point_rdd, rectangle_rdd, False, True)

In [ ]:
# without passing column names, the result will contain only two geometries columns
geometry_df = Adapter.toDf(result, sedona)

In [ ]:
geometry_df.printSchema()

In [ ]:
geometry_df.show(5)

In [ ]:
geometry_df.collect()[0]

## Passing column names

In [ ]:
geometry_df = Adapter.toDf(result, ["left_user_data"], ["right_user_data"], sedona)

In [ ]:
geometry_df.show(5)

# Converting RDD Spatial join result to DF directly, avoiding jvm python serde

In [ ]:
query_envelope = Envelope(-85.01, -60.01, 34.01, 50.01)

result_range_query = RangeQueryRaw.SpatialRangeQuery(linestring_rdd, query_envelope, False, False)

In [ ]:
# converting to df
gdf = Adapter.toDf(result_range_query, sedona)

In [ ]:
gdf.show(5)

In [ ]:
gdf.printSchema()

In [ ]:
# Passing column names
# converting to df
gdf_with_columns = Adapter.toDf(result_range_query, sedona, ["_c1"])

In [ ]:
gdf_with_columns.show(5)

In [ ]:
gdf_with_columns.printSchema()

# Summary

We have shown how to install Sedona with Pyspark and run a basic example (source: https://github.com/apache/sedona/blob/master/binder/ApacheSedonaCore.ipynb) on Google Colab.

In [ ]:
print("Chu Hữu Đăng Trường")